In [69]:
import pandas as pd
import numpy as np

In [70]:
# peek csv
df = pd.read_csv('data/output_file.csv', parse_dates=['date'])
df = df.astype({'ticker': 'string'})
df.to_csv('test.csv', header=True)
df.head()

,date,volume,open,high,low,close,adjclose,ticker
0,2018-06-15,3696200,59.630001,60.080002,59.369999,60.009998,60.009998,TMUS
1,2018-07-26,23518700,43.580002,43.830002,43.220001,43.529999,41.350727,CSCO
2,2018-04-13,2828300,108.849998,109.459999,108.519997,109.260002,101.975182,PEP
3,2018-03-14,21141100,45.340000,45.759998,45.090000,45.279999,42.340096,CSCO
4,2019-05-21,2506000,277.089996,279.779999,274.649994,275.329987,275.329987,AVGO


In [71]:
df.groupby('ticker').size().reset_index()

,ticker,0
0,AAPL,503
1,AMD,503
2,AMZN,503
3,AVGO,503
4,CSCO,503
5,MSFT,503
6,NFLX,503
7,PEP,503
8,TMUS,503
9,TSLA,503


In [72]:
a = df.groupby('ticker')
len(a)


10

In [73]:
df.groupby('ticker').get_group('AAPL').sort_values('date').reset_index()

,index,date,volume,open,high,low,close,adjclose,ticker
0,1364,2018-01-02,25555900,170.160004,172.300003,169.259995,172.259995,166.353714,AAPL
1,3380,2018-01-03,29517900,172.529999,174.550003,171.960007,172.229996,166.324722,AAPL
2,4875,2018-01-04,22434600,172.539993,173.470001,172.080002,173.029999,167.097290,AAPL
3,2124,2018-01-05,23660000,173.440002,175.369995,173.050003,175.000000,168.999741,AAPL
4,4171,2018-01-08,20567800,174.350006,175.610001,173.929993,174.350006,168.372040,AAPL
...,...,...,...,...,...,...,...,...,...
498,534,2019-12-24,12119700,284.690002,284.890015,282.920013,284.269989,282.831299,AAPL
499,1093,2019-12-26,23280300,284.820007,289.980011,284.700012,289.910004,288.442780,AAPL
500,1969,2019-12-27,36566500,291.119995,293.970001,288.119995,289.799988,288.333313,AAPL
501,2554,2019-12-30,36028600,289.459991,292.690002,285.220001,291.519989,290.044617,AAPL


In [74]:
df.groupby(['ticker', 'date']).first()

volume        open        high         low       close  \
ticker date                                                                   
AAPL   2018-01-02  25555900  170.160004  172.300003  169.259995  172.259995   
       2018-01-03  29517900  172.529999  174.550003  171.960007  172.229996   
       2018-01-04  22434600  172.539993  173.470001  172.080002  173.029999   
       2018-01-05  23660000  173.440002  175.369995  173.050003  175.000000   
       2018-01-08  20567800  174.350006  175.610001  173.929993  174.350006   
...                     ...         ...         ...         ...         ...   
TSLA   2019-12-24   8054700  418.359985  425.470001  412.690002  425.250000   
       2019-12-26  10633900  427.910004  433.480011  426.350006  430.940002   
       2019-12-27   9945700  435.000000  435.309998  426.109985  430.380005   
       2019-12-30  12586400  428.790009  429.000000  409.260010  414.700012   
       2019-12-31  10285700  405.000000  421.290009  402.079987  418.329987   

                     adjclose  
ticker date                    
AAPL   2018-01-02  166.353714  
       2018-01-03  166.324722  
       2018-01-04  167.097290  
       2018-01-05  168.999741  
       2018-01-08  168.372040  
...                       ...  
TSLA   2019-12-24  425.250000  
       2019-12-26  430.940002  
       2019-12-27  430.380005  
       2019-12-30  414.700012  
       2019-12-31  418.329987  

[5030 rows x 6 columns]

In [134]:
f = {'open': 'first', 'close': 'last', 'high': 'max', 'low': 'min'}
monthly = df.sort_values('date').groupby(['ticker', pd.Grouper(key='date', freq='ME')], observed=True).agg(f).reset_index()#.drop(columns=['date'])
monthly.to_csv('inter/monthly.csv', header=True)
monthly.head()

,ticker,date,open,close,high,low
0,AAPL,2018-01-31,170.160004,167.429993,180.100006,164.699997
1,AAPL,2018-02-28,167.169998,178.119995,180.619995,150.240005
2,AAPL,2018-03-31,178.539993,167.779999,183.500000,164.940002
3,AAPL,2018-04-30,166.639999,165.259995,178.940002,160.630005
4,AAPL,2018-05-31,166.410004,186.869995,190.369995,165.270004


In [135]:
prev_ema = -1

def ema(x, window):
    global prev_ema
    cur_price = x[-1]
    mult = 2 / (window + 1)
    ema = (cur_price - prev_ema) * mult + prev_ema
    prev_ema = ema
    return ema

def add_indicators(monthly: pd.DataFrame):
    global prev_ema
    monthly['sma_10'] = monthly['close'].rolling(window=10).mean()
    monthly['sma_20'] = monthly['close'].rolling(window=20).mean()
    prev_ema = monthly['sma_10'].dropna().iloc[0]
    monthly['ema_10'] = monthly['close'].rolling(window=10).apply(lambda x: ema(x, 10), raw=True)
    prev_ema = monthly['sma_20'].dropna().iloc[0]
    monthly['ema_20'] = monthly['close'].rolling(window=20).apply(lambda x: ema(x, 20), raw=True)
    display(monthly)
    monthly.to_csv('temp2.csv')

add_indicators(monthly[0:24].copy())

,ticker,date,open,close,high,low,sma_10,sma_20,ema_10,ema_20
0,AAPL,2018-01-31,170.160004,167.429993,180.100006,164.699997,NaN,NaN,NaN,NaN
1,AAPL,2018-02-28,167.169998,178.119995,180.619995,150.240005,NaN,NaN,NaN,NaN
2,AAPL,2018-03-31,178.539993,167.779999,183.500000,164.940002,NaN,NaN,NaN,NaN
3,AAPL,2018-04-30,166.639999,165.259995,178.940002,160.630005,NaN,NaN,NaN,NaN
4,AAPL,2018-05-31,166.410004,186.869995,190.369995,165.270004,NaN,NaN,NaN,NaN
5,AAPL,2018-06-30,187.990005,185.110001,194.199997,180.729996,NaN,NaN,NaN,NaN
6,AAPL,2018-07-31,183.820007,190.289993,195.960007,183.419998,NaN,NaN,NaN,NaN
7,AAPL,2018-08-31,199.130005,227.630005,228.869995,197.309998,NaN,NaN,NaN,NaN
8,AAPL,2018-09-30,228.410004,225.740005,229.669998,215.300003,NaN,NaN,NaN,NaN
9,AAPL,2018-10-31,227.949997,218.860001,233.470001,206.089996,191.308998,NaN,196.318271,NaN


In [150]:
prev_ema = -1

def sma(df, window):
    return df.rolling(window=window).mean()

def add_sma(df, col, periods):
    for period in periods:
        df[f'sma_{period}'] = sma(df[col], period)
    return df

def ema(x, window):
    global prev_ema
    cur_price = x[-1]
    mult = 2 / (window + 1)
    ema = (cur_price - prev_ema) * mult + prev_ema
    prev_ema = ema
    return ema

def add_ema(df, col, periods):
    global prev_ema

    for period in periods:
        prev_ema = sma(df[col], period).dropna().iloc[0]
        df[f'ema_{period}'] = df[col].rolling(window=period).apply(lambda x: ema(x, period), raw=True)
        
    return df

def add_techincal_indicators(monthly: pd.DataFrame):

    monthly = add_sma(monthly, 'close', [10, 20])
    monthly = add_ema(monthly, 'close', [10, 20])

    display(monthly)
    monthly.to_csv('temp2.csv')

add_techincal_indicators(monthly[0:24].copy())

,ticker,date,open,close,high,low,sma_10,sma_20,ema_10,ema_20
0,AAPL,2018-01-31,170.160004,167.429993,180.100006,164.699997,NaN,NaN,NaN,NaN
1,AAPL,2018-02-28,167.169998,178.119995,180.619995,150.240005,NaN,NaN,NaN,NaN
2,AAPL,2018-03-31,178.539993,167.779999,183.500000,164.940002,NaN,NaN,NaN,NaN
3,AAPL,2018-04-30,166.639999,165.259995,178.940002,160.630005,NaN,NaN,NaN,NaN
4,AAPL,2018-05-31,166.410004,186.869995,190.369995,165.270004,NaN,NaN,NaN,NaN
5,AAPL,2018-06-30,187.990005,185.110001,194.199997,180.729996,NaN,NaN,NaN,NaN
6,AAPL,2018-07-31,183.820007,190.289993,195.960007,183.419998,NaN,NaN,NaN,NaN
7,AAPL,2018-08-31,199.130005,227.630005,228.869995,197.309998,NaN,NaN,NaN,NaN
8,AAPL,2018-09-30,228.410004,225.740005,229.669998,215.300003,NaN,NaN,NaN,NaN
9,AAPL,2018-10-31,227.949997,218.860001,233.470001,206.089996,191.308998,NaN,196.318271,NaN
